In [ ]:
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
tfk = tf.keras
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers

In [ ]:
import tensorflow as tf
tfk = tf.keras
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir = '../input/trainingdata3/training'

In [ ]:
image_generator_train = tfk.preprocessing.image.ImageDataGenerator(rotation_range = 75,
                                                             height_shift_range = 50,
                                                             width_shift_range= 50,
                                                             zoom_range = 0.3,
                                                             horizontal_flip = True,
                                                             vertical_flip=True,
                                                             fill_mode='reflect',
                                                             validation_split=0.1,
                                                             preprocessing_function=tfk.applications.xception.preprocess_input)

train_data = image_generator_train.flow_from_directory(dataset_dir,
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = True,
                                                 seed = seed,
                                                 subset = "training")


#For validation is necessary to create an ImageDataGenerator without data augmentation
image_generator_validation = tfk.preprocessing.image.ImageDataGenerator(preprocessing_function=tfk.applications.xception.preprocess_input, 
                                                             validation_split=0.1)


validation_data = image_generator_validation.flow_from_directory(dataset_dir,
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = False,
                                                 seed = seed,
                                                 subset = "validation")

In [ ]:
input_shape = (256,256,3)

In [ ]:
#load pre trained xception with fully connected with transfer learning
fine_tuning_model = tfk.models.load_model('../input/xception-right2/CNN_TLxception_model')
fine_tuning_model.summary()

In [ ]:
#choose how many layer of the pretrained net train
def set_trainable_layers(num_layers, model):
    
    for layer in model.layers[:-num_layers]:
        layer.trainable = False
    
    for layer in model.layers[-num_layers:]:
        layer.trainable = True
        

set_trainable_layers(60,fine_tuning_model ) #ultimi 60 a true

In [ ]:
#callbacks (earlystopping)
callbacks = []
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor ='val_loss', patience=2, restore_best_weights=True))

In [ ]:
# Compile the model
fine_tuning_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
#Train the model
history = fine_tuning_model.fit(x = train_data,
                        validation_data= validation_data,
                        epochs = 30,
                        callbacks = callbacks).history



In [ ]:
fine_tuning_model.save("./CNN_FTXCEPTIONAugmentedDataset_model")
#zip the folder in order to be able to dowload it
!zip -r file4.zip "./CNN_FTXCEPTIONAugmentedDataset_model"